**Entorno:** Anaconda (Python)

_Notebook preparado para ejecutarse en un entorno Anaconda Python local._

---



**BASE DE DATOS III - TAREA N°2**

*Resumen de la tarea*

**Integrantes:**
- Rodrigo Guerrero
- Miguel Espinoza

---

# **DESCRIPCIÓN DEL DATASET**

Conjunto de datos oficiales que recopila indicadores hospitalarios de establecimientos públicos y privados de Chile, correspondientes a procesos de hospitalización registrados por el Ministerio de Salud (MINSAL).
- **Tamaño original**: 155339 filas, 20 columnas
- **Origen**: Evaluación empírica con información de un conjunto de datos hospitalario de Chile


---



# ***VARIABLES DEL DATASET***

**Variable objetivo**
- EFICIENCIA: Esta columna indica si un hospital es eficiente o no (1 = Sí, 0 = No).


**Variables independientes**
- TIPO_PERTENENCIA: Código numérico que identifica la pertenencia del establecimiento (tipo de dato: Entero)

- GLOSA_SSS: Nombre del Servicio de Salud (tipo de dato: Texto)

- PERIODO: Año del registro (tipo de dato: Entero)

- ESTABLECIMIENTO: Nombre del establecimiento (tipo de dato: Texto)

- AREA_FUNCIONAL: Nombre del área funcional (tipo de dato: Texto)

- DIAS_CAMAS_OCUPADAS: Total de las camas ocupadas durante el periodo (tipo de dato: Entero)

- DIAS_CAMAS_DISPONIBLES: Total de días que las camas estuvieron disponibles durante el periodo (tipo de dato: Entero)

- DIAS_ESTADA: Suma de los días de estadía de todos los pacientes hospitalarios durante el periodo (tipo de dato: Entero)

- NUMERO_EGRESOS: Total de pacientes que egresaron del hospital (tipo de dato: Entero)

- MES: Mes que se realizó el registro (tipo de dato: Entero)

- EGRESOS_FALLECIDOS: Número de pacientes que fallecieron durante hospitalización (tipo de dato: Entero)

- TRASLADOS: Cantidad de egresos que corresponden a pacientes trasladados a otro centro (tipo de dato: Entero)

- INDICE_OCUPACIONAL: Proporción de camas ocupadas respecto a las disponibles (tipo de dato: Decimal)

- PROMEDIO_CAMAS_DISPONIBLES: Promedio de camas disponibles (tipo de dato: Decimal)

- PROMEDIO_DIAS_ESTADA: Días que un paciente permanece hospitalizado (tipo de dato: Decimal)

- LETALIDAD: Porcentaje de fallecidos respecto al total de egresos (tipo de dato: Decimal)

- INDICE_ROTACION: Número promedio de egresos por cama durante el periodo (tipo de dato: Decimal)

- COD_SSS: Código numérico que identifica al Servicio de Salud al que pertenece un establecimiento hospitalario (tipo de dato: Entero)

- CODIGO_ESTABLECIMIENTO: Código único que identifica a cada establecimiento de salud dentro del sistema (tipo de dato: Entero)

- COD_AREA_FUNCIONAL: Código numérico que corresponde al área funcional del hospital o centro de salud (tipo de dato: Entero)


---



### LIBRERÍAS UTILIZADAS

- **pandas**: Manipulación y análisis de datos estructurados
- **numpy**: Operaciones numéricas y arrays multidimensionales
- **matplotlib.pyplot**: Creación de gráficos y visualizaciones
- **seaborn**: Visualizaciones estadísticas avanzadas


In [ ]:
#librerias
import pandas as pd  # Pandas para manipulación de datos
import matplotlib.pyplot as plt  # Matplotlib para visualización de datos
import seaborn as sns  # Seaborn para gráficos estadísticos
import numpy as np  # NumPy para operaciones numéricas
import re

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier  # K-Vecinos más cercanos
from sklearn.linear_model import LogisticRegression  # Regresión Logística
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix,precision_score  # Métricas de rendimiento
from sklearn.model_selection import train_test_split  # División de datos
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA  # Reducción de dimensionalidad
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, train_test_split


# **1. a. Descripción utilizando métodos estadísticos**

## **Descripcion detallada del dataset**

In [ ]:
# Cargar el dataset
csv_path = r"indicadores_rem20_20250925.csv"
df = pd.read_csv(csv_path, na_values=["nan ", ""], thousands=",", quotechar='"', on_bad_lines='skip', delimiter=';')

# Ver las primeras filas
df

## **Variable Eficiencia**

**Justificación de uso**

- La variable EFICIENCIA se construye como un indicador compuesto que refleja el desempeño operativo y clínico de los hospitales incluidos en el dataset. Se consideraron tres dimensiones fundamentales:

- Índice Ocupacional (INDICE_OCUPACIONAL):
Representa la proporción de camas efectivamente utilizadas en relación con la capacidad total. Se define un umbral de 0.7, considerando que un hospital que mantiene más del 70% de ocupación logra un uso adecuado de sus recursos, evitando subutilización excesiva de camas y personal.


- Letalidad (LETALIDAD):
Se mide como la proporción de pacientes fallecidos sobre el total de egresos. Para efectos de eficiencia, se considera favorable un nivel inferior al 5% (0.05), asumiendo que hospitales con menor letalidad combinan atención oportuna y calidad clínica.


- Índice de Rotación (INDICE_ROTACION):
Este índice refleja la velocidad con que las camas se desocupan y se ocupan nuevamente, indicando la capacidad de gestión de flujos de pacientes. Se establece que una eficiencia mayor se asocia a valores superiores a la mediana del dataset, lo que implica un manejo más dinámico y eficiente de los recursos hospitalarios.

In [ ]:
# EFICIENCIA basada en rangos razonables
df['EFICIENCIA'] = (
    (df['INDICE_OCUPACIONAL'] > 0.7) &
    (df['LETALIDAD'] < 0.05) &
    (df['INDICE_ROTACION'] > df['INDICE_ROTACION'].median())
).astype(int)


# Revisar primeras filas
print(df[['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION', 'EFICIENCIA']].head())

# Contar cuántos 1 y 0 hay
Distribucion_Clases = df['EFICIENCIA'].value_counts()
Proporcion_Clases = df['EFICIENCIA'].value_counts(normalize=True) * 100

print("\nDistribución de Clases:")
print(Distribucion_Clases)

print("\nProporción de Clases:")
print(Proporcion_Clases)



## **Balanceo**

-Se utilizó SMOTE porque el conjunto de datos presenta un desbalance de clases (73%-27%) entre hospitales eficientes y no eficientes. Esta técnica genera nuevos ejemplos sintéticos de la clase minoritaria a partir de sus vecinos más cercanos, lo que permite equilibrar las clases sin eliminar información y mejorar el rendimiento de los modelos de clasificación, especialmente con variables numéricas continuas como las de este caso.

In [ ]:

X = df[['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']]
y = df['EFICIENCIA']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(y.value_counts())
print(y_train_res.value_counts())



## Tipo de datos de cada columna


In [ ]:
print(df.dtypes)

In [ ]:
df_numerico = df.select_dtypes(include=['number'])

## **Medidas Descriptivas**

In [ ]:
descripcion_estadistica = df_numerico.describe()
print(descripcion_estadistica)

## **Media**

In [ ]:
media = df_numerico.mean()
print(f"Media:\n{media}")

## **Medianas**

In [ ]:
mediana = df_numerico.median()
print(f"Mediana:\n{mediana}")

## **Moda**

In [ ]:
moda = df_numerico.mode().iloc[0]
print(f"Moda:\n{moda}")

## **Desviacion Estandar**

In [ ]:
desviacion_estandar = df_numerico.std()
print(f"Desviación Estándar:\n{desviacion_estandar}")

## **Percentiles**

In [ ]:
percentiles = df_numerico.quantile(q=[0.1, 0.25, 0.5, 0.75, 0.9])
print(f"Percentiles:\n{percentiles}")

In [ ]:
# Detección de valores atípicos usando el rango intercuartílico

Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1

# Identificar outliers
outliers = ((df_numerico < (Q1 - 1.5 * IQR)) | (df_numerico > (Q3 + 1.5 * IQR))).sum()
print(f"Outliers detectados por columna:\n{outliers}")


# **1. b. Visualización de los datos**

In [ ]:
# Indice ocupacional
df['INDICE_OCUPACIONAL'] = pd.to_numeric(df['INDICE_OCUPACIONAL'], errors='coerce')

plt.hist(df['INDICE_OCUPACIONAL'].dropna(), bins=10, alpha=0.7, color='steelblue')
plt.xlabel('Índice Ocupacional')
plt.ylabel('Frecuencia')
plt.title('Distribución del Índice Ocupacional')
plt.show()

# Letalidad
df['LETALIDAD'] = pd.to_numeric(df['LETALIDAD'], errors='coerce')

plt.hist(df['LETALIDAD'].dropna(), bins=10, alpha=0.7, color='tomato')
plt.xlabel('Letalidad')
plt.ylabel('Frecuencia')
plt.title('Distribución de la Letalidad')
plt.show()

# Indice de rotacion
df['INDICE_ROTACION'] = pd.to_numeric(df['INDICE_ROTACION'], errors='coerce')

plt.hist(df['INDICE_ROTACION'].dropna(), bins=10, alpha=0.7, color='seagreen')
plt.xlabel('Índice de Rotación')
plt.ylabel('Frecuencia')
plt.title('Distribución del Índice de Rotación')
plt.show()

In [ ]:
# Boxplot de Índice Ocupacional por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'INDICE_OCUPACIONAL'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'INDICE_OCUPACIONAL'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Índice Ocupacional')
plt.title('Índice Ocupacional según Eficiencia')
plt.show()

# Boxplot de Letalidad por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'LETALIDAD'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'LETALIDAD'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Letalidad')
plt.title('Letalidad según Eficiencia')
plt.show()

# Boxplot de Índice de Rotación por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'INDICE_ROTACION'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'INDICE_ROTACION'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Índice de Rotación')
plt.title('Índice de Rotación según Eficiencia')
plt.show()

In [ ]:
# Asegurar tipo numérico
df['INDICE_OCUPACIONAL'] = pd.to_numeric(df['INDICE_OCUPACIONAL'], errors='coerce')
df['LETALIDAD'] = pd.to_numeric(df['LETALIDAD'], errors='coerce')

# Separar por eficiencia
eficiente = df[df['EFICIENCIA'] == 1]
no_eficiente = df[df['EFICIENCIA'] == 0]

plt.scatter(no_eficiente['INDICE_OCUPACIONAL'], no_eficiente['LETALIDAD'], alpha=0.5, label='No Eficiente', color='red')
plt.scatter(eficiente['INDICE_OCUPACIONAL'], eficiente['LETALIDAD'], alpha=0.5, label='Eficiente', color='green')
plt.xlabel('Índice Ocupacional')
plt.ylabel('Letalidad')
plt.title('Relación entre Índice Ocupacional y Letalidad según Eficiencia')
plt.legend()
plt.show()


# **1. c. Exploración, limpieza y transformación de datos**

In [ ]:
# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
display(df.head())

# Información general del dataset
print("\nInformación general del dataset:")
print(df.info())

# Descripción estadística de las variables numéricas
print("\nDescripción estadística de las variables numéricas:")
display(df.describe().T)


# Revisión de valores únicos en columnas categóricas
print("\nValores únicos en columnas categóricas:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].nunique()} valores únicos")


In [ ]:
# Contar valores nulos totales
print("Valores nulos totales antes de limpiar:")
print(df.isnull().sum().sum())

# Eliminar filas con valores nulos en variables relevantes
variables_relevantes = ['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']
df_limpio = df.dropna(subset=variables_relevantes)

print(f"\nFilas originales: {len(df)}, Filas después de eliminar nulos: {len(df_limpio)}")

# Detección de valores atípicos usando el rango intercuartílico (IQR)
for col in variables_relevantes:
    Q1 = df_limpio[col].quantile(0.25)
    Q3 = df_limpio[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df_limpio = df_limpio[(df_limpio[col] >= limite_inferior) & (df_limpio[col] <= limite_superior)]

print(f"Filas después de eliminar outliers: {len(df_limpio)}")

# Volver a calcular estadísticas con los datos limpios
print("\nDescripción estadística después de limpiar los datos:")
display(df_limpio.describe().T)


In [ ]:
# Copiar el dataset limpio
df_transformado = df_limpio.copy()

# Transformar variables categóricas en numéricas (Label Encoding)
columnas_categoricas = ['GLOSA_SSS', 'AREA_FUNCIONAL', 'ESTABLECIMIENTO']
le = LabelEncoder()
for col in columnas_categoricas:
    if col in df_transformado.columns:
        df_transformado[col] = le.fit_transform(df_transformado[col].astype(str))

print("\nColumnas categóricas transformadas a numéricas correctamente.")

# Estandarización de variables numéricas
columnas_numericas = ['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']
scaler = StandardScaler()
df_transformado[columnas_numericas] = scaler.fit_transform(df_transformado[columnas_numericas])

print("Variables numéricas estandarizadas correctamente.")

# Verificar el resultado final
print("\nVista general de los datos transformados:")
display(df_transformado.head())


In [ ]:
# ==========================
# Punto 2 - Clasificación
# ==========================

# Preparar datos (usar df_transformado si existe)
try:
    df_used = df_transformado.copy()
except NameError:
    try:
        df_used = df_limpio.copy()
    except NameError:
        df_used = df.copy()

features = ['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']
for f in features:
    df_used[f] = pd.to_numeric(df_used[f], errors='coerce')

# Eliminar filas con NaN en features o target
df_used = df_used.dropna(subset=features + ['EFICIENCIA'])
X = df_used[features]
y = df_used['EFICIENCIA'].astype(int)

# Revisar distribución de clases
print('Distribución de clases (dataset):')
print(y.value_counts())
print('\nProporción (%):')
print(y.value_counts(normalize=True)*100)

# Dividir en train/test con estratificación
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar SMOTE si hay desbalance > 70/30
from imblearn.over_sampling import SMOTE
imbalance_ratio = y.value_counts(normalize=True).max()
apply_smote = imbalance_ratio > 0.7
if apply_smote:
    print('\nDesbalance detectado (>70/30). Aplicando SMOTE sobre el conjunto de entrenamiento...')
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    print('Distribución después de SMOTE:')
    print(y_train.value_counts())
else:
    print('\nNo se aplicó SMOTE (balance aceptable).')

# Modelos a evaluar
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
models = {
    'KNN': KNeighborsClassifier(),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42)
}

# Parámetros para GridSearch (ligero)
param_grids = {
    'KNN': {'n_neighbors': [3,5,7]},
    'AdaBoost': {'n_estimators': [50,100]},
    'RandomForest': {'n_estimators': [50,100], 'max_depth': [5, None]}
}

best_estimators = {}
results = {}
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for name, model in models.items():
    print(f"\nEntrenando y ajustando: {name}")
    grid = GridSearchCV(model, param_grids[name], scoring='roc_auc', cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best = grid.best_estimator_
    best_estimators[name] = best
    print(f"Mejor params {name}: {grid.best_params_}")
    # Predicciones y métricas
    y_pred = best.predict(X_test)
    if hasattr(best, 'predict_proba'):
        y_proba = best.predict_proba(X_test)[:,1]
    else:
        # algunos clasificadores usan decision_function
        try:
            y_proba = best.decision_function(X_test)
        except Exception:
            y_proba = None
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | ROC AUC: {auc:.4f} if available")
    print('\nClassification report:')
    print(classification_report(y_test, y_pred, zero_division=0))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_pred))
    results[name] = {'model': best, 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'auc': auc}

# Curvas ROC comparadas
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc as calc_auc
plt.figure(figsize=(8,6))
for name, info in results.items():
    model = info['model']
    if hasattr(model, 'predict_proba') or hasattr(model, 'decision_function'):
        if hasattr(model, 'predict_proba'):
            prob = model.predict_proba(X_test)[:,1]
        else:
            prob = model.decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, prob)
        roc_auc = calc_auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.3f})")
plt.plot([0,1],[0,1],'k--', alpha=0.6)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Model comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Importancia de variables (si RandomForest)
if 'RandomForest' in best_estimators:
    rf = best_estimators['RandomForest']
    try:
        importances = rf.feature_importances_
        fi = pd.Series(importances, index=features).sort_values(ascending=False)
        print('\nImportancia de variables (RandomForest):')
        print(fi)
        fi.plot(kind='bar', title='Feature importance - RandomForest')
        plt.show()
    except Exception as e:
        print('No se pudo obtener importancia de características:', e)

# Guardar el mejor modelo por AUC
best_by_auc = sorted([(k,v['auc']) for k,v in results.items() if v['auc'] is not None], key=lambda x: x[1] if x[1] is not None else -1, reverse=True)
if best_by_auc:
    best_name = best_by_auc[0][0]
    print(f"\nMejor modelo por AUC: {best_name} (AUC={best_by_auc[0][1]:.3f})")
    best_model = results[best_name]['model']
else:
    # fallback por F1
    best_name = sorted(results.items(), key=lambda x: x[1]['f1'], reverse=True)[0][0]
    best_model = results[best_name]['model']
    print(f"\nMejor modelo por F1: {best_name}")

print('\nPunto 2 completado: modelos entrenados y métricas generadas.')